In [1]:
import numpy as np
import gensim
import PySimpleGUI as sg
import pickle
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.utils import resample
import pandas as pd

ModuleNotFoundError: No module named 'gensim'

In [ ]:
with open('total_data.pkl','rb') as f:
    total_data = pickle.load(f)

In [2]:
def doc2vec(list lst):
    new = 0
    count = 0
    for word in lst:
        new += model.wv(word)
        count += 1
    return new/count
    

SyntaxError: invalid syntax (<ipython-input-2-470cb3f2ee63>, line 1)

In [ ]:
list_x = []
list_y = []
dataset = []
for key in total_data.keys():
    # fitting paper into tfidf vector
    for paragraph in total_data[key]:
        if paragraph[-1] == 1:
            dataset.append(doc2vec(paragraph[0].split()))
            if paragraph[-2] is False:
                list_y.append(0)
            else:
                list_y.append(1)

In [ ]:
y_train = np.asarray(list_y)
len_train = int(len(y_train)*0.9)
X, X_test = arr[:len_train], arr[len_train:]
y, Y_test = y_train[:len_train], y_train[len_train:]
print(X.shape)
print(y.shape)

In [ ]:
def over_sampler(X,y):
    count = np.sum(y)
    idxx = np.argwhere(y==1)[:,0]
    new_idxx = np.random.choice(idxx, np.sum(y==0)- np.sum(y))
    X_0 = np.concatenate([X, X[new_idxx]],axis=0)
    y_0 = np.concatenate([y, y[new_idxx]])
    return X_0, y_0

In [ ]:
X_train, Y_train = over_sampler(X, y)
print(X_train.shape)
print(Y_train.shape)

In [ ]:
model = LogisticRegression()
model.fit(X_train, Y_train)

In [ ]:
y_pred = model.predict(X_train)
print(f1_score(Y_train, y_pred))
cm = confusion_matrix(Y_train, y_pred)
print(cm)
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
Y_pred = model.predict(X_test)
print(f1_score(Y_test, Y_pred))
cm = confusion_matrix(Y_test, Y_pred)
print(cm)
plt.matshow(cm)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()